# NYC SIRD Notebook 

In [1]:
import math
import pandas as pd
import cufflinks as cf
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
from scipy.optimize import curve_fit
from scipy.integrate import odeint
from typing import List
from IPython.display import display
from datetime import timedelta
cf.go_offline()

## Load datasets

See [other notebooks in this repo](https://github.com/jpoles1/open_covid19) for code which can help with scraping these datasets.

In [2]:
italy_province_data = pd.read_csv("output/italy_province_report_data.csv")
print("Loaded Italy province data!")
display(italy_province_data.head(1))
italy_region_data = pd.read_csv("output/italy_region_report_data.csv")
print("Loaded Italy region data!")
display(italy_region_data.head(1))
jhu_data = pd.read_csv("output/daily_jhu_report_data.csv")
print("\nLoaded JHU data!")
display(jhu_data.head(1))
nytimes_data = pd.read_csv("output/nytime_data.csv")
print("\nLoaded NYTimes data!")
display(nytimes_data.head(1))

Loaded Italy province data!


,date,data,stato,codice_regione,region,codice_provincia,province,sigla_provincia,lat,long,total positive cases,note_it,note_en
0,2020-02-24,2020-02-24T18:00:00,ITA,13,Abruzzo,69,Chieti,CH,42.351032,14.167546,0.0,NaN,NaN


Loaded Italy region data!


,date,data,stato,codice_regione,region,lat,long,hospitalized with symptoms,terapia_intensiva,total hospitalized,in home isolation,total currently positive,total newly positive,recovered,deaths,total positive cases,tamponi,note_it,note_en
0,2020-02-24,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,0,0,0,0,0,5,NaN,NaN



Loaded JHU data!


,Date,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,FIPS,Admin2,Active,Combined_Key
0,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Loaded NYTimes data!


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0


## Processing data for analysis

This data analysis pipeline takes in one dataframe per locale. This dataframe should be indexed by date with a column for each of "Confirmed", "Recovered", and "Deaths". See example after next cell.

_Note: in light of the formatting of available data at the time of authoring, this analysis pipeline takes in counts of total cases confirmed (rather than # of currently infected individuals)._

In [3]:
nytimes_nyc_data = nytimes_data[nytimes_data["county"] == "New York City"][["date", "cases", "deaths"]]
nytimes_nyc_data["Recovered"] = 0
nytimes_nyc_data = nytimes_nyc_data[["date", "cases", "Recovered", "deaths"]] 
nytimes_nyc_data.columns = ["Date", "Confirmed", "Recovered", "Deaths"]
nytimes_nyc_data.set_index("Date", inplace=True)

#JHU data seems more complete for Chinese provinces
jhu_hubei_data = jhu_data[jhu_data["Province/State"] == "Hubei"][["Date", "Confirmed", "Recovered", "Deaths"]]
jhu_hubei_data["Date"] = pd.to_datetime(jhu_hubei_data["Date"])
jhu_hubei_data.set_index("Date", inplace=True)

jhu_hunan_data = jhu_data[jhu_data["Province/State"] == "Hunan"][["Date", "Confirmed", "Recovered", "Deaths"]]
jhu_hunan_data["Date"] = pd.to_datetime(jhu_hunan_data["Date"])
jhu_hunan_data.set_index("Date", inplace=True)

#Italian data for regions
lombardia_data = italy_region_data[italy_region_data["region"] == "Lombardia"][["date", "total positive cases", "recovered", "deaths"]]
lombardia_data.columns = ["Date", "Confirmed", "Recovered", "Deaths"]
lombardia_data.set_index("Date", inplace=True)

emilia_romagna_data = italy_region_data[italy_region_data["region"] == "Emilia Romagna"][["date", "total positive cases", "recovered", "deaths"]]
emilia_romagna_data.columns = ["Date", "Confirmed", "Recovered", "Deaths"]
emilia_romagna_data.set_index("Date", inplace=True)

In [4]:
#Example clean data
nytimes_nyc_data.tail(3)

,Confirmed,Recovered,Deaths
Date,,,
2020-03-25,20011,0,280
2020-03-26,23112,0,365
2020-03-27,25399,0,450


In [5]:
#Collect clean locale data into dictionary for batch processing
locale_data = {
    "NYC": nytimes_nyc_data, 
    "Hubei": jhu_hubei_data, 
    "Lombardia": lombardia_data,
}
#Population count required for each locale
pop_counts = {"NYC": 8623000, "Hubei": 58500000, "Lombardia": 10040000}
#Hospital bed counts 
# NYC count from: https://www.bloomberg.com/graphics/2020-new-york-coronavirus-outbreak-how-many-hospital-beds/
locale_hospital_bed_dict = {"NYC": 23000, "Hubei": None, "Lombardia": None}

## Ready to Start With SIRD Modeling!

With your clean data in hand you can start using the sird_reg() function as a jumping-off point for your SIRD parameter fitting and modeling!

In [6]:
#Setup boundary conditions for non-linear least-squared optimization (beta_0, t_beta, gamma, delta)
ls_bounds = ([0, 5, 0.02, 0.001], [1, 60, 1, 0.25]);
nyc_plot_data = None;
def sird_reg(locale_data, locale_pop, locale_name, t_lockdown = None, t_lift = None,
             hospital_beds = None, t_beta_override = None, 
             ls_bounds=ls_bounds, forecast_days: int = 5, show_ci: bool = False):
    """
    Performs non-linear least-squares fitting of SIRD parameters on a dataframe
    Attributes:
        locale_data: dataframe = df with columns ["Confirmed", "Recovered", "Deaths"] and date index 
        locale_pop: int = total population of the locale from which this dataframe is derived
        locale_name: str = name of locale (for tables and plots)
        t_lockdown: datetime, t_lift: datetime = datetime of when social-distancing begins and ends, None to ignore
        hospital_beds: int: number of hospital beds, if known
        t_beta_override = If set, will not attempt to fit t-beta using least squared, None to fit t_beta
        ls_bounds: ([int], [int]) =least-squared regression boundary condition for each parameter ([beta_0_lower, t_beta_lower...], [beta_0_upper, t_beta_upper...])
        forecast_days: int = number of days past input data to forecast values
        show_ci: bool = shows confidence intervals (TODO: not currently implemented)
    """
    locale_sird = locale_data.copy()
    #Start by converting total confirmed cases to SIRD variables
    locale_sird["Susceptible"] = locale_pop - locale_sird["Confirmed"]
    locale_sird["Infected"] = locale_sird["Confirmed"] - locale_sird["Recovered"] - locale_sird["Deaths"] #- locale_sird["Recovered"]
    locale_sird.drop("Confirmed", axis=1, inplace=True)
    locale_sird = locale_sird[["Susceptible", "Infected", "Recovered", "Deaths"]]
    #Fill NaNs with 0
    locale_sird.fillna(0, inplace=True)
    #Remove rows from prior to first recorded infection
    date_list = locale_sird.index[locale_sird["Infected"] > 1] #for plotting later
    locale_sird = locale_sird[locale_sird["Infected"] > 1]
    if(t_lockdown != None):
        t_lockdown = (t_lockdown - pd.to_datetime(date_list[0])).days
    if(t_lift != None):
        t_lift = (t_lift - pd.to_datetime(date_list[0])).days
    #Reset index to days since first recorded infection
    locale_sird.reset_index(drop=True, inplace=True)

    def sird_fn(timepoints, beta_0, t_beta, gamma, delta):
        """
        Define function which will be used for non-linear least-sq fitting 
        Attributes:
            timepoints = timepoints at which to calculate integral of ODE
            beta_0 = initial infxn coeff | t_beta=infxn coeff decay period 
            gamma = recovery coeff | delta = death coeff | x = timepoint
        Returns:
            Flattened integral of [S, I, R, D] at timepoints using initial values from locale_sird
            Must be reshaped below before plotting
        """
        def sird_ode(y_t, t):
            """
            Define system of ODEs to be integrated for SIRD model
            Attributes:
                y_t = y(t) = [S, I, R, D] 
                t = timepoint
            Returns:
                Derivative of y(t) at timepoint t = [dS(t)/dt, dI(t)/dt, dR(t)/dt, dD(t)/dt] 
            """
            
            #beta attenuates over the course of the outbreak
            beta = beta_0
            if t_lift != None and t > t_lift:
                beta = beta_0 * math.exp(-(t - (t_lockdown if t_lockdown else 0)) / (t_beta*2 if t_beta_override == None else t_beta_override*2))
            elif t_lockdown == None or t > t_lockdown:
                beta = beta_0 * math.exp(-(t - (t_lockdown if t_lockdown else 0)) / (t_beta if t_beta_override == None else t_beta_override))
            ds_dt = -beta * y_t[0] * y_t[1] / locale_pop #Susceptible
            dd_dt = delta * y_t[1] #Deaths
            dr_dt = gamma * y_t[1] #Recovered
            di_dt = -ds_dt - dd_dt - dr_dt #Infected
            return [ds_dt, di_dt, dr_dt, dd_dt]
        #Calculate the integral of the above ODE over the range of timepoints using first data point as initial value
        sird_int = odeint(sird_ode, locale_sird.values[0], timepoints)
        return sird_int.ravel()
    #Use non-linear least-squares optimization to fit parameters to known data
    param_est, param_cov = curve_fit(sird_fn, locale_sird.index.values, locale_sird.values.ravel(),
                                     method="trf", bounds=ls_bounds)
    #compute 1x StDev errors for params using diagonal of covariance matrix
    param_stdev = np.sqrt(np.diag(param_cov))
    #Predict SIRD using model, shape will be 1 x (4*# predicted)
    predicted_data = sird_fn(range(len(locale_sird.index) + forecast_days), *param_est)
    #Reshape into (# predicted, 4)
    predicted_data = predicted_data.reshape(len(locale_sird.index) + forecast_days, 4)
    #Rename columns properly
    predicted_data = pd.DataFrame(predicted_data, columns=["Susceptible", "Infected", "Recovered", "Deaths"])
    #Construct plot dataframe
    plot_data = predicted_data[["Infected", "Recovered", "Deaths"]]
    plot_data.columns = ["Projected Infected", "Projected Recovered", "Projected Deaths"]
    plot_data = plot_data.join(locale_sird[["Infected", "Recovered", "Deaths"]])
    if hospital_beds != None:
        plot_data["Projected Hospitalized"] = plot_data["Projected Infected"] * 0.2 #20% of infected estimated to require hospitalization
    #Convert index back from days since outbreak to dates
    plot_data.index = pd.date_range(date_list[0], pd.to_datetime(date_list[-1]) + timedelta(days=forecast_days)).to_pydatetime()
    #Change t_beta param_est/stdev if using override
    param_est[1] = param_est[1] if t_beta_override == None else t_beta_override
    param_stdev[1] = param_stdev[1] if t_beta_override == None else 0
    #Plot
    plot_modes = {col: ("markers" if "Projected" in col else "lines") for col in plot_data.columns}
    fig = plot_data.iplot(asFigure = True, mode=plot_modes, size=4, width=3, colorscale="plotly",
                          title="SIRD Model of %s | Beta_0=%s | T_beta=%s | Gamma=%s | Delta=%s" % (locale_name, param_est[0].round(2), param_est[1].round(1), param_est[2].round(4), param_est[3].round(4)))
    if hospital_beds != None:
        fig.add_shape(
            type="line",
            x0=plot_data.index[0],
            y0=hospital_beds,
            x1=plot_data.index[-1],
            y1=hospital_beds,
            line=dict(
                color="Red",
                width=3,
                dash="dashdot",
            ),
        )
    if t_lockdown != None:
        fig.add_shape(
            type="line",
            x0=plot_data.index[t_lockdown],
            y0=0,
            x1=plot_data.index[t_lockdown],
            y1=plot_data.max().max(),
            line=dict(
                color="Dark Blue",
                width=3,
                dash="dot",
            ),
        )
    if t_lift != None:
        fig.add_shape(
            type="line",
            x0=plot_data.index[t_lift],
            y0=0,
            x1=plot_data.index[t_lift],
            y1=plot_data.max().max(),
            line=dict(
                color="Red",
                width=3,
                dash="dashdot",
            ),
        )
    fig.show()
    return (param_est, param_stdev, plot_data)
#Collect output from each locales SIRD regression
sird_param_est = []
sird_param_stdev = []
sird_rmse = []
#Define the manual t_beta overrides. If None, will optimize this parameter instead.
#Override useful for early data when quarantine measures not yet or still in process.
t_beta_dict = {"Hubei": None, "NYC": None, "Lombardia": 21}
#Define the day from onset of outbreak at which lockdown goes into effect. If None, will ignore.
t_lockdown_dict = {"Hubei": "01-23-2020", "NYC": "03-19-2020", "Lombardia": "03-09-2020"}
#Define the day from onset of outbreak at which lockdown is lifted. If None, will ignore.
t_lift_dict = {"Hubei": "03-27-2020", "NYC": "04-19-2020", "Lombardia": None}
#Iterate over dictionary of locales and their clean dataframes (created above)
for (locale, locale_df) in locale_data.items(): 
    #Run regression and receive parameter estimates/stdevs
    res = sird_reg(locale_df, pop_counts[locale], locale, forecast_days=180, t_beta_override=t_beta_dict[locale],
                  t_lockdown = pd.to_datetime(t_lockdown_dict[locale]), t_lift = pd.to_datetime(t_lift_dict[locale]), 
                   hospital_beds = locale_hospital_bed_dict[locale])
    #Store parameter estimates and stdev for display later
    sird_param_est.append(res[0])
    sird_param_stdev.append(res[1])
    #Calculate root mean square error (RMSE) for each outcome variable prediction (SIRD)
    plot_data = res[2].dropna()
    if locale == "NYC":
        #save NYC plot data for use outside loop
        nyc_plot_data = plot_data
    infected_rmse = math.sqrt(np.sum((plot_data["Infected"]-plot_data["Projected Infected"]).pow(2))/len(plot_data.index))
    recovered_rmse = math.sqrt(np.sum((plot_data["Recovered"]-plot_data["Projected Recovered"]).pow(2))/len(plot_data.index))
    death_rmse = math.sqrt(np.sum((plot_data["Deaths"]-plot_data["Projected Deaths"]).pow(2))/len(plot_data.index))
    sird_rmse.append([infected_rmse, recovered_rmse, death_rmse])
print("SIRD Regression Parameter Estimates")
display(pd.DataFrame(sird_param_est, index=locale_data.keys(), columns=["Beta_0", "T_beta", "Gamma", "Delta"]))
sird_param_95ci = [zip(np.maximum([0, 0, 0, 0], x - 2*sird_param_stdev[i]).round(5), np.maximum([0, 0, 0, 0], x + 2*sird_param_stdev[i]).round(5)) for i,x in enumerate(sird_param_est)]
print("\n\nSIRD Regression Parameter 95% Confidence Intervals")
display(pd.DataFrame(sird_param_95ci, index=locale_data.keys(), columns=["Beta_0", "T_beta", "Gamma", "Delta"]))
print("\n\nSIRD Outcome RMSEs")
display(pd.DataFrame(sird_rmse, index=locale_data.keys(), columns=["Infected RMSE", "Recovered RMSE", "Deaths RMSE"]))

SIRD Regression Parameter Estimates


,Beta_0,T_beta,Gamma,Delta
NYC,0.476007,7.356359,0.020000,0.001000
Hubei,0.520137,10.798873,0.039837,0.002310
Lombardia,0.271733,21.000000,0.040245,0.025738




SIRD Regression Parameter 95% Confidence Intervals


,Beta_0,T_beta,Gamma,Delta
NYC,"(0.46992, 0.4821)","(6.87104, 7.84168)","(0.01469, 0.02531)","(0.0, 0.00618)"
Hubei,"(0.50091, 0.53937)","(10.18673, 11.41102)","(0.03745, 0.04222)","(0.00096, 0.00366)"
Lombardia,"(0.26827, 0.2752)","(21.0, 21.0)","(0.03735, 0.04314)","(0.02292, 0.02855)"




SIRD Outcome RMSEs


,Infected RMSE,Recovered RMSE,Deaths RMSE
NYC,492.605735,644.217874,107.017500
Hubei,7575.072245,5999.468888,483.651240
Lombardia,939.985235,360.584505,202.911125


## Hospitalization Stats Via Bloomberg vs. Predicted Hospitalizations

In [7]:
hospitalizations = pd.read_csv("https://www.bloomberg.com/graphics/2020-new-york-coronavirus-outbreak-how-many-hospital-beds/data/nyc_daily_cases.csv")
hospitalizations.rename(columns={"date": "Date", "hosp": "Hospitalized"}, inplace=True)
hospitalizations.set_index("Date", inplace=True)
hospitalizations[["Hospitalized"]].join(nyc_plot_data[["Projected Hospitalized"]]).dropna()

,Hospitalized,Projected Hospitalized
Date,,
2020-03-19,554.0,580.245769
2020-03-22,1160.0,1758.798053
2020-03-24,2883.0,2933.687618
2020-03-25,3922.0,3597.922600


## References:
- [Chinese and Italian COVID-19 outbreaks can be correctly described by a modified SIRD model](https://www.medrxiv.org/content/10.1101/2020.03.19.20039388v1)

Future reading:
- [Analysis and forecast of COVID-19 spreading in China, Italy and France](https://www.sciencedirect.com/science/article/pii/S0960077920301636)
- [Preliminary analysis of COVID-19 spread in Italy with an adaptive SEIRD model](https://arxiv.org/pdf/2003.09909.pdf)
- [Tracing DAY-ZERO and Forecasting the Fade out of the COVID-19 Outbreak in Lombardy, Italy: A Compartmental Modelling and Numerical Optimization Approach.](https://www.medrxiv.org/content/10.1101/2020.03.17.20037689v2)
- [Estimation of the final size of the COVID-19 epidemic](https://www.medrxiv.org/content/10.1101/2020.02.16.20023606v5.full.pdf+html)